# Import Lib

In [1]:
import pandas as pd
import numpy as np


# Load Data and Add Deposit Ratio Bin

In [ ]:

df = pd.read_csv("cleaned_loan_data.csv")


# Bin Deposit Ratio
ratio_bins = [0, 0.2, 0.4, 0.6, 0.8, np.inf]
ratio_labels = ['<20%', '20-40%', '40-60%', '60-80%', '80%+']
df['deposit_ratio_bin'] = pd.cut(df['depositratio'], bins=ratio_bins, labels=ratio_labels, right=False)
print("Data loaded. Shape:", df.shape) 


Data loaded. Shape: (10000, 13)


# Segment & aggregate

In [3]:
# Segment categories and aggregate by loan performance
segment_summary = (
    df.groupby(['age_bucket','cartype','area', 'deposit_ratio_bin'])
    .agg(
        applications=('funded','count'),
        approvals=('applicationoutcome', lambda x: (x==1).sum()),
        approval_rate=('applicationoutcome', lambda x: (x==1).mean()),
        conversion_rate=('funded','mean'),
        avg_loan=('loanamount','mean'),
        total_funded=('loanamount', lambda x: x[df.loc[x.index,'funded']==1].sum())
    )
    .reset_index()
)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_16684\1365657554.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['age_bucket','cartype','area', 'deposit_ratio_bin'])


# Rank segments

In [4]:
import math

n = 25  # Top and bottom n segments
rank = math.ceil(n * len(segment_summary)/100)

# Sort by total funded loans
top_segments = segment_summary.sort_values('total_funded', ascending=False).head(n)
bottom_segments = segment_summary.sort_values('total_funded', ascending=True).head(n)

print("🔝 Top (25%) High-Value Segments")
print(top_segments)

print("\n🔻 Bottom (25%) Low-Value Segments")
print(bottom_segments)


🔝 Top (25%) High-Value Segments
    age_bucket      cartype   area deposit_ratio_bin  applications  approvals  \
95         55+  Convertible  urban              <20%           223      215.0   
65       45-54  Convertible  urban              <20%           191      169.0   
90         55+  Convertible  rural              <20%           203      179.0   
36       35-44  Convertible  urban            20-40%           141      121.0   
105        55+          SUV  urban              <20%           137      131.0   
60       45-54  Convertible  rural              <20%           194      146.0   
30       35-44  Convertible  rural              <20%           176      139.0   
5        25-34  Convertible  urban              <20%           161       83.0   
35       35-44  Convertible  urban              <20%           152      114.0   
96         55+  Convertible  urban            20-40%           112      108.0   
91         55+  Convertible  rural            20-40%           129      101.0